# TO DO 
- [x] Responsivity - bootstrap for metric
- [ ] Mean spike rate per trial
- [x] Tuning Width
- [x] Tuning Peak
- [x] Smooth tuning curves
- [ ] Running vs not

Notes:
- [x] Distribution-based metrics of responsivity (bootstrapping, etc) over arbitrary values (Baden 2016 style)
- [ ] Population measures
    - [ ] distribution of widths/prefs
    - [ ] pop. vector decoding

In [ ]:
%load_ext autoreload
%autoreload 2

# Imports
import numpy as np
import pandas as pd
import xarray as xr
import panel as pn
import holoviews as hv
import datashader as dshade

import os
import sys
import random
import importlib
import datetime
import warnings
import math
import cmath
import pycircstat as circ
warnings.filterwarnings('ignore')

from pprint import pprint
from itertools import zip_longest
from scipy.stats import sem, norm, binned_statistic, percentileofscore, ttest_1samp, ttest_ind
from scipy.optimize import least_squares

from bokeh.resources import INLINE
from bokeh.io import export_svgs, export_png
from holoviews import opts, dim
from holoviews.operation import histogram
from holoviews.operation.datashader import datashade, shade
hv.extension('bokeh')
# hv.extension('matplotlib')
import matplotlib.pyplot as plt
%matplotlib notebook

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
%aimport paths
import processing_parameters
import functions_bondjango as bd
import functions_plotting as fp
import functions_data_handling as fdh
import functions_kinematic as fk

importlib.reload(fp)
# set up the figure theme
fp.set_theme()

# Functions

## Utilities

In [ ]:
def normalize_rows(data_in):
    
    for idx, el in enumerate(data_in):
        data_in[idx, :] = (el-np.nanmin(el))/(np.nanmax(el)-np.nanmin(el))
    return data_in

def normalize(data_in):
    return (data_in - np.nanmin(data_in)) / (np.nanmax(data_in) - np.nanmin(data_in))

## Gaussian Fitting

In [ ]:
import functions_tuning as tuning

## Plotting

In [ ]:
def spike_raster(data, cells=None):
        if cells is None:
            cells = [el for el in data.columns if 'cell' in el]
    
        spikes = data.loc[:, cells]
    
        im = hv.Image((data.time_vector, np.arange(len(cells)), spikes.values.T), 
                        kdims=['Time (s)', 'Cells'], vdims=['Activity (a.u.)'])
        im.opts(width=600) #, cmap='Purples')
        return im
    
def trace_raster(data, cells=None, ds_factor=1):
    if cells is None:
        cells = [el for el in data.columns if 'cell' in el]

    trace = data.loc[:, cells]
    max_std = trace.std().max()

    lines = {i: hv.Curve((data.time_vector, trace.iloc[:, i].values.T + i*max_std)) for i in np.arange(len(cells))}
    lineoverlay = hv.NdOverlay(lines, kdims=['Time (s)']).opts(height=500, width=800)
    return lineoverlay


def plot_tuning_curve(tuning_curve, **kwargs):
    plt.plot(*tuning_curve)
    
    if "fit" in kwargs:
        plt.plot(*kwargs['fit'])
        
    if "pref_angle" in kwargs:
        plt.axvline(kwargs['pref_angle'], color='k', linewidth=1)
        
def plot_tuning_curve_hv(tuning_curve, fit=None, sem=None, trials=None, pref_angle=None, **kwargs):
    overlay = []
    
    tuning = hv.Curve(tuning_curve).opts(width=600, height=300, **kwargs)
    overlay.append(tuning)
    
    if fit is not None:
        fit_plot = hv.Curve(fit)
        overlay.append(fit_plot)
        
    if sem is not None:
        sem_plot = hv.Spread((*tuning_curve, sem)).opts(fill_alpha=0.25)
        overlay.append(sem_plot)
        
    if trials is not None:
        trials_plot = hv.Scatter(trials).opts(color='r', size=3)
        overlay.append(trials_plot)
        
    if pref_angle is not None:
        pref_plot = hv.VLine(pref_angle).opts(color='k', line_width=1)
        overlay.append(pref_plot)
                  
    return hv.Overlay(overlay)

# Load the Data

In [ ]:
importlib.reload(processing_parameters)

# get the search string
search_string = processing_parameters.search_string + r", analysis_type:preprocessing"

# get the paths from the database
file_path, paths_all, parsed_query, date_list, animal_list = fdh.fetch_preprocessing(search_string)

animal_idxs = [i for i,d in enumerate(animal_list) if d==parsed_query['mouse'].lower()]
good_entries = [file_path[index] for index in animal_idxs]
input_path = [paths_all[index] for index in animal_idxs]

# # assemble the output path
print(input_path)

In [ ]:
from ast import literal_eval

class Cell():
    def __init__(self, name):
        self.id = name

class WirefreeExperiment():
    def __init__(self, filepath, use_xarray=False):
        # Experiment attributes
        self.animal = None
        self.rig = None
        self.exp_type = None
        self.exp_date = None
        self.exp_params = None

        # Experimental metadata
        self.cell_matches = None
        self.arena_corners = None
        
        self.roi_info = None
        self.trial_set = None
        self.full_kinematics = None

        # Neural and kinematic data structures
        self.kinematics = None
        self.raw_spikes = None
        self.raw_fluor = None

        # Load the data
        self._load_hdf(filepath, use_xarray)
        self.cells = [el for el in self.raw_spikes.columns if "cell" in el]
        self._create_cell_class()


    def _load_hdf(self, file, use_xarray):
        with pd.HDFStore(file) as h:

                self._parse_params(h['params'])
                self._add_orientation_to_params()
            
                self._parse_kinematic_data(h['matched_calcium'], use_xarray)

                if use_xarray:
                    self.full_kinematics = h['full_traces'].to_xarray()
                else:
                    self.full_kinematics = h['full_traces']

                self.roi_info = h['roi_info']
                self.trial_set = h['trial_set']
                self.cell_matches = h['cell_matches']
                self.arena_corners = h['arena_corners']

    def _parse_kinematic_data(self, matched_calcium, use_xarray):

        # Calculate orientation expicitly
        if 'orientation' not in matched_calcium.columns:
            matched_calcium['orientation'] = matched_calcium['direction']
            matched_calcium['orientation'][(matched_calcium['orientation'] > -180) & (matched_calcium['orientation'] < 0)] += 180
           
        # For all data
        self.rig = matched_calcium['mouse'].values[0].split('_', 1)[0]
        if self.rig in ['VWheel', 'VWheelWF']:
            self.exp_type = 'fixed'
        else:
            self.exp_type = 'free'
        
        self.animal = matched_calcium['mouse'].values[0].split('_', 1)[1:]
        self.exp_date = matched_calcium['datetime'][0]
        spikes_cols = [key for key in matched_calcium.keys() if 'spikes' in key]
        fluor_cols = [key for key in matched_calcium.keys() if 'fluor' in key]
        motive_tracking_cols = ['mouse_y_m','mouse_z_m','mouse_x_m','mouse_yrot_m','mouse_zrot_m','mouse_xrot_m']


        # If there is more than one spatial or temporal frequency, include it, othewise don't
        stimulus_cols = ['trial_num', 'time_vector', 'direction', 'orientation', 'grating_phase',]
        if len(self.exp_params['temporal_freq']) > 1:
            stimulus_cols.append('temporal_freq')
        if len(self.exp_params['spatial_freq']) > 1:
            stimulus_cols.append('spatial_freq')
        
        # For headfixed data
        eye_cols = ['eye_horizontal_vector_x', 'eye_horizontal_vector_y', 'eye_midpoint_x', 'eye_midpoint_y', 'pupil_center_ref_x', 'pupil_center_ref_y', 'fit_pupil_center_x', 'fit_pupil_center_y',
                       'pupil_diameter', 'minor_axis', 'pupil_rotation', 'eyelid_angle']
        eye_dlc_cols = ['pupil_center_x','pupil_center_y','pupil_top_left_x','pupil_top_left_y','pupil_top_x','pupil_top_y','pupil_top_right_x','pupil_top_right_y' 'pupil_right_x' 'pupil_right_y', 
                           'pupil_bottom_right_x','pupil_bottom_right_y','pupil_bottom_x','pupil_bottom_y','pupil_bottom_left_x','pupil_bottom_left_y','pupil_left_x','pupil_left_y','eye_corner_nasal_x',
                           'eye_corner_nasal_y','eye_corner_temporal_x','eye_corner_temporal_y','eyelid_top_x','eyelid_top_y','eyelid_bottom_x','eyelid_bottom_y','led_x','led_y']
        wheel_cols = ['wheel_speed', 'wheel_acceleration']
        
        # For free data
        mouse_kinem_cols = ['mouse_heading', 'mouse_angular_speed', 'mouse_speed', 'mouse_acceleration', 'head_direction', 'head_height']
        mouse_dlc_cols = ['mouse_snout_x', 'mouse_snout_y', 'mouse_barl_x', 'mouse_barl_y', 'mouse_barr_x', 'mouse_barr_y', 'mouse_x', 'mouse_y', 'mouse_body2_x', 'mouse_body2_y', 
                          'mouse_body3_x', 'mouse_body3_y', 'mouse_base_x', 'mouse_base_y', 'mouse_head_x', 'mouse_head_y', 'miniscope_top_x', 'miniscope_top_y']

        if self.exp_type == 'fixed':
            self.kinematics = matched_calcium.loc[:, stimulus_cols + motive_tracking_cols + eye_cols + wheel_cols]
        else:
            self.kinematics = matched_calcium.loc[:, stimulus_cols + motive_tracking_cols + mouse_kinem_cols]
            
        self.raw_spikes = matched_calcium.loc[:, stimulus_cols + spikes_cols]
        self.raw_spikes.columns = [key.rsplit('_', 1)[0] if 'spikes' in key else key for key in self.raw_spikes.columns]
        self.raw_fluor = matched_calcium.loc[:, stimulus_cols + fluor_cols]
        self.raw_fluor.columns = [key.rsplit('_', 1)[0] if 'fluor' in key else key for key in self.raw_fluor.columns]

        if use_xarray:
            self.kinematics = self.kinematics.to_xarray()
            self.raw_spikes = self.raw_spikes.to_xarray()
            self.raw_fluor = self.raw_fluor.to_xarray()

    def _parse_params(self, df):
        params = df.to_dict('list')
        for key in params.keys():
            try:
                params[key] = np.array(literal_eval(params[key][0]))
            except:
                params[key] = params[key][0]
        
        self.exp_params = params

    def _add_orientation_to_params(self):
        self.exp_params['direction'].sort()
        self.exp_params['orientation'] = self.exp_params['direction'].copy()
        self.exp_params['orientation'][self.exp_params['orientation'] < 0] += 180
        self.exp_params['orientation'].sort()

        # Add wrapped directions, useful for plotting
        directions_wrapped = np.sort(fk.wrap(self.exp_params['direction']))
        directions_wrapped = np.concatenate((directions_wrapped, [360.]))
        self.exp_params['directions_wrapped'] = directions_wrapped

    def _create_cell_class(self):
        self.cell_props_spikes = {}
    
        for cell in self.cells:
            self.cell_props_spikes[cell] = Cell(cell)

In [ ]:
fixed = WirefreeExperiment(input_path[0], use_xarray=False)
free = WirefreeExperiment(input_path[1], use_xarray=False)

# Remove baselines from the data for later processing
Also normalize inferred spike responses

In [ ]:
def normalize_spike_responses(ds, quantile=0.07):
    ds_norm = ds.fillna(0)
    
    # get the number of cells
    if type(ds) is pd.DataFrame:
        cells = [el for el in ds.columns if "cell" in el]
        # Normalize cell responses across all sessions
        ds_norm[cells].apply(normalize)
    
        # Get the 7th percentile of activity per cell for each stimulus
        # Try 7th/8th percentile
        percentiles = ds_norm.groupby(['direction'])[cells].quantile(quantile)
    
        # get the baselines - The first row is the inter-trial interval
        baselines = percentiles.iloc[0, :]
    
        # Subtract baseline from everything
        ds_norm[cells].subtract(baselines, axis=1)

    elif type(ds) == xr.Dataset:
        cells = [el for el in ds.data_vars if "cell" in el]
    

    return ds_norm

In [ ]:
free.norm_spikes = normalize_spike_responses(free.raw_spikes)
fixed.norm_spikes = normalize_spike_responses(fixed.raw_spikes)

# Extract matched cells

In [ ]:
matched_data = []
unmatched_data = []
remap_dict = []
for i, ds in enumerate(data):
    cells = np.array([el for el in ds.columns if "cell" in el])
    cell_nums = np.unique([int(cell.split("_")[1])  for cell in cells])
    
    keep = np.in1d(cell_nums, cell_matches.iloc[:, i])
    match_cells = cells[np.argwhere(keep)].flatten()
    unmatch_cells = cells[np.argwhere(~keep)].flatten()
    
    matched_ds = ds.drop(unmatch_cells, axis="columns")
    matched_data.append(matched_ds)
    
    unmatched_ds = ds.drop(match_cells, axis="columns")
    unmatched_data.append(unmatched_ds)

old_name_0 = []
old_name_1 = []
rename_0 = []
rename_1 = []
for i, match_pair in enumerate(cell_matches.values):
    new_number = f'{i:04}'
    cells_0 = [el for el in matched_data[0].columns if f"cell_{match_pair[0]:04}" in el]
    old_name_0.extend(cells_0)
    cells_1 = [el for el in matched_data[1].columns if f"cell_{match_pair[1]:04}" in el]
    old_name_1.extend(cells_1)
    
    new_cells_0 = [el.replace(f"{match_pair[0]:04}", new_number) for el in cells_0]
    rename_0.extend(new_cells_0)
    new_cells_1 = [el.replace(f"{match_pair[1]:04}", new_number) for el in cells_1]
    rename_1.extend(new_cells_1)
    
rename_dict_0 = dict(zip(old_name_0, rename_0)) 
rename_dict_1 = dict(zip(old_name_1, rename_1)) 

matched_data[0].rename(columns=rename_dict_0, inplace=True)
matched_data[1].rename(columns=rename_dict_1, inplace=True)

# Determine responsivity

In [ ]:
ds = matched_data[0]
# get the number of cells
cell_spikes = [el for el in ds.columns if 'spikes' in el]

# Get response per trial for each cell
for cell in cell_spikes:
    trial_responses = ds[ds.trial_num >= 1].groupby(['trial_num'])[cell].mean().to_numpy()
    isi_responses = ds[ds.trial_num == 0][cell].mean()
#     isi_responses = np.ones(trial_responses.shape) * isi_responses
    
    ttest = ttest_ind(isi_responses, trial_responses)

# trial_start = ds[ds.trial_num >= 1].groupby(['trial_num']).time_vector.nth(0)
# trial_end = ds[ds.trial_num >= 1].groupby(['trial_num']).time_vector.nth(-2)
# trial_times = np.array((trial_start, trial_end)).T

In [ ]:
isi_responses

# Some simple exploration

## Plot activity

In [ ]:
# Inferred spikes
spike_plots = [trace_raster(free.norm_spikes), trace_raster(fixed.norm_spikes)]
hv.Layout(spike_plots).cols(2).opts(shared_axes=False)

In [ ]:
# allocate memory for the plots
fluor_plots = [trace_raster(free.raw_fluor), trace_raster(fixed.raw_fluor)]
hv.Layout(fluor_plots).cols(2).opts(shared_axes=False)

In [ ]:
# average across repeats and time


## Visualize the activity for all units during the inter-trial interval

In [ ]:
free_iti = free.norm_spikes.loc[free.norm_spikes['trial_num'] == 0, ['time_vector'] + free.cells].reset_index(drop=True)
trace_raster(free_iti).opts(ylabel="Cells", width=900, height=500)

In [ ]:
fixed_iti = fixed.norm_spikes.loc[fixed.norm_spikes['trial_num'] == 0, ['time_vector'] + fixed.cells].reset_index(drop=True)
trace_raster(fixed_iti).opts(ylabel="Cells", width=900, height=500)

## Plot the activity of each cell 

In [ ]:
direction_plots = []
for cell in free.cells[:2]:
    for direction in free.exp_params['direction']:
        individual_responses = free.norm_spikes.groupby(['trial_num'])[cell].agg(list)
        resp_array = np.array(list(zip_longest(*individual_responses, fillvalue=np.NaN))).T
        current_direction = np.nanmean(resp_array, axis=0)
        current_sem = sem(resp_array, axis=0, nan_policy='omit')

        x = np.arange(resp_array.shape[-1])
        plot = hv.Curve(current_direction).opts(ylabel=str(direction), xlabel="time")
        plot2 = hv.Spread((x, current_direction, current_sem))
        plot.opts(xrotation=45)
        direction_plots.append(plot*plot2)

hv.Layout(direction_plots).opts(shared_axes=False).cols(len(free.exp_params['direction']))

## Plot direction selectivity

In [ ]:
def plot_trial_responses(ds, key, cells=None):
    # allocate memory for the cell plots
    cell_plots = []

    if cells is None:
        cells = [el for el in data.columns if 'cell' in el]
    num_cells = len(cells)
    
    tuning = ds.groupby([key])[cells].mean()
    tuning_sem = ds.groupby([key])[cells].sem()
    percentiles = ds.groupby([key])[cells].quantile(0.07)

    # get the baselines - The first column is the inter-trial interval
    baselines = percentiles.iloc[0, percentiles.columns.get_loc(cells[0]):percentiles.columns.get_loc(cells[-1])+1]

    # Subtract baseline from everything
    ds[cells].subtract(baselines, axis=1)
    
    # Get the mean reponse on each trial
    trial_responses = ds.groupby([key, 'trial_num'])[cells].agg(np.nanmean)
    # Drop the trial number level and regroup by orientation
    trial_responses = trial_responses.droplevel(['trial_num']).groupby([key]).agg(list)
    
    for cell in cells:
        
        # get the current cell dActivity and sem
        current_cell = tuning.iloc[1:, tuning.columns.get_loc(cell)]
        current_sem = tuning_sem.iloc[1:, tuning_sem.columns.get_loc(cell)]
        
        # get the current cell dActivity
        current_cell_trials = trial_responses.iloc[1:, trial_responses.columns.get_loc(cell)].to_list()
        current_cell_trials = list_lists_to_array(current_cell_trials)

        # get the orientation or direction
        label = current_cell.index.to_numpy()
        label_by_observation = np.multiply(np.ones(current_cell_trials.shape), label[:, np.newaxis])
        
        # Create a 2D array of positions and values for scatter plot
        X = np.vstack((np.ravel(label_by_observation), np.ravel(current_cell_trials))).T

        # plot
        if key == 'direction':
            x_lim = (-180, 180)
        else:
            x_lim= (0, 180)
            
        plot_scatter = hv.Scatter(X).opts(color='r', size=3, xlabel=key, xlim=x_lim, xrotation=45)
        plot_mean = hv.Curve((label, current_cell))
        plot_sem = hv.Spread((label, current_cell.values, current_sem))
        cell_plots.append(plot_scatter*plot_mean*plot_sem)
        
    return cell_plots

In [ ]:
# allocate memory for the cell plots
cell_plots = []

# for all the files
for ds in matched_data:
    ds_plots = plot_trial_responses(ds, 'direction')
    cell_plots.append(ds_plots)
        
num_cells = len(cell_plots[0])
cell_plots = sum(cell_plots, [])
hv.Layout(cell_plots).opts(shared_axes=False).cols(num_cells)

## Plot orientation selectivity

In [ ]:
# allocate memory for the cell plots
cell_plots = []

# for all the files
for ds in matched_data:
    ds_plots = plot_trial_responses(ds, 'orientation')
    cell_plots.append(ds_plots)
        
num_cells = len(cell_plots[0])
cell_plots = sum(cell_plots, [])
hv.Layout(cell_plots).opts(shared_axes=False).cols(num_cells)

# SVD tuning analysis (Baden et al 2016)

## SVD Helpers

In [ ]:
def map_binned_statistic(x, statistic=np.nanmean):
    vals_to_be_binned = np.array(x)
    x_array = np.arange(0, len(vals_to_be_binned))
    bin_means, bin_edges, bin_numbers = binned_statistic(x_array, vals_to_be_binned, statistic=statistic, bins=25)
    return bin_means

def mean_histogram(x):
    hists = np.array(x)
    hist_mean = np.mean(hists, axis=0)
    
    # Normalize mean
    hist_mean = hist_mean / np.nanmax(hist_mean)
    return hist_mean


def generate_mean_response_matrix(cell_responses, tuning_kind):
    agg_current_cell = cell_responses.groupby([tuning_kind]).agg(lambda x: [h for h in x])

    # Take the mean across binned trials
    current_cell_mean_resp = agg_current_cell.apply(mean_histogram)
    
    # Get the sem across binned trials
    agg_current_cell.apply(mean_histogram)

    # Get a list of directions/orientations
    angles = current_cell_mean_resp.index.to_numpy()

    # Wrap angles from [-180, 180] to [0, 360] for direction tuning, and sort angles
    # Needed for pycircstat toolbox
    angles_wrapped = fk.wrap(angles)
    sort_idx = np.argsort(angles_wrapped)
    angles_sorted = angles_wrapped[sort_idx]

    # Create the response matrix (direction x time response)
    response_mat = np.vstack(current_cell_mean_resp.values)

    # Sorted to match sorted angles
    sorted_response_mat = response_mat[sort_idx, :]

    # Make sure to explicitly represent 360 degrees in the data if looking at direction tuning. 
    # This is done by duplicating the 0 degrees trace
    if tuning_kind == 'direction':
        idx_0 = np.argwhere(angles_sorted == 0)[0][0]
        trace_360 = sorted_response_mat[idx_0, :]
        angles_sorted = np.append(angles_sorted, 360.)
        sorted_response_mat = np.append(sorted_response_mat, [trace_360], axis=0)

    # Get into time, direction format
    response_mat = sorted_response_mat.T
    
    return response_mat, angles_sorted

def svd_selectivity_idx(response_mat, angles, tuning_kind, verbose=False):
    # Note the @ is matrix multiplication in Python 3.x
    
    # Zero out nans so that the SVD converges
    response_mat = np.nan_to_num(response_mat, nan=0.0)
    
    # Do SVD. Note that in numpy's implementation, V is already transposed, 
    # so M ~ U @ np.diag(S) @ V 
    U, S, V = np.linalg.svd(response_mat, full_matrices=False)

    # Some sanity checks
    if verbose:
        print(f'Dimensions of U: {U.shape}, S: {S.shape}, V: {V.shape}')
        print(f'Is the SVD approximation close? {np.allclose(response_mat, U @ np.diag(S) @ V)}')

    # Compute direction/orientation selectivity and DSi/OSi
    tuning_curve = V.T[:, 1] # Transpose V because of numpy svd convention
    temporal_component = U[:, 1]

    if tuning_kind == 'direction':
        im_exp = 1j
    else:
        im_exp = 2j

    phi = np.exp(im_exp * np.deg2rad(angles))
    phi = np.expand_dims(phi, axis=-1)
    K = phi.T @ tuning_curve
    selectivity_idx = np.absolute(K)
    
    return selectivity_idx, tuning_curve, temporal_component

def bootstrap_svd_selectivity(trial_responses, tuning_kind, num_shuffles=1000):
    
    trial_idxs = trial_responses.index.to_numpy()
    shuffled_selectivity_idxs = []

    for i in range(0, num_shuffles):
        # Shuffle stimulus labels and reassign
        np.random.shuffle(trial_idxs)
        shuffled_responses = trial_responses.copy().reindex(trial_idxs)

        # Generate normalized mean response matrix
        response_mat, angles_sorted = generate_mean_response_matrix(shuffled_responses, tuning_kind)

        # Do SVD
        selectivity_idx, _, _ = svd_selectivity_idx(response_mat, angles_sorted, tuning_kind)

        shuffled_selectivity_idxs.append(selectivity_idx)

    shuffled_selectivity_idxs = np.concatenate(shuffled_selectivity_idxs)
    
    return shuffled_selectivity_idxs

## Main Loop

In [ ]:
keys = ['direction', 'orientation']

# Allocate memory for cell property dataframes
df_list = []

for i, (ds, exp) in enumerate(zip(matched_data, exp_type)):

    # get the number of cells
    cells = [el for el in ds.columns if 'spike' in el]
    
    # allocate memory for the experiment
    exp_list = []

    for key in keys:
        key_abbr = key[:3]

        # Get the mean reponse during each time bin on each trial
        trial_responses = ds.groupby([key, 'trial_num'])[cells].agg(list)
        trial_responses = trial_responses.applymap(map_binned_statistic)

        # Drop the trial number level, remove inter-trial interval from df
        trial_responses = trial_responses.droplevel(['trial_num'])
        trial_responses = trial_responses.drop(trial_responses[trial_responses.index < -500].index)

        df_data_list = []

        for cell in cells:
            
            # Aggregate trials for each stimulus
            current_cell_responses = trial_responses.iloc[:, trial_responses.columns.get_loc(cell)]

            # Generate normalized mean response matrix
            response_mat, angles_sorted = generate_mean_response_matrix(current_cell_responses, key)

            # Do SVD to get DSi/OSi
            selectivity_idx, tuning_curve, temporal_component = svd_selectivity_idx(response_mat, angles_sorted, key)

            # Normalize tuning curve to its range
            tuning_curve = normalize(tuning_curve)
            
            #-- Get response quality index of the cell --#
            response_mat = np.nan_to_num(response_mat, nan=0.0)
            QI = np.var(np.mean(response_mat, axis=1)) / np.mean(np.var(response_mat, axis=0))
            
            #-- Run permutation test --#
            # Here we shuffle the trial IDs and compare the real selectivity index to the bootstrapped distribution
            shuffled_selectivity_idxs = bootstrap_svd_selectivity(current_cell_responses, key, num_shuffles=300)
            p = percentileofscore(shuffled_selectivity_idxs, selectivity_idx, kind='mean') / 100.
            
            #-- Fit Gaussian tuning curves to get preference and tuning width --#
            if key == "direction":
                fit, fit_curve, pref_angle, real_pref_angle = calculate_pref_direction(angles_sorted, tuning_curve)
            else:
                fit, fit_curve, pref_angle, real_pref_angle = calculate_pref_orientation(angles_sorted, tuning_curve)

            # Get tuning width.
            tuning_width = get_FWHM(fit.x[2])
            
            # Append all the data for saving
            df_data_list.append([cell, float(selectivity_idx), p, QI, pref_angle, tuning_width, (angles_sorted, tuning_curve), fit_curve])

        # Make data frame for the experiment
        cell_props = pd.DataFrame(data=df_data_list, 
                                  columns=['cell', f'{key_abbr}_sel_idx', f'{key_abbr}_sel_p', f'{key_abbr}_qi', f'{key_abbr}_pref', f'{key_abbr}_width', f'{key_abbr}_tuning_curve', f'{key_abbr}_fit_curve'],
                                      )
        cell_props.insert(0, 'exp_type', ''.join([exp, str(i)]))
        exp_list.append(cell_props)
        
    df_list.append(exp_list[0].merge(exp_list[1], on=['cell', 'exp_type']))

In [ ]:
cell_properties_svd = df_list[0]

In [ ]:
bins = np.linspace(0, max(selectivity_idx[0], np.max(shuffled_selectivity_idxs)), retstep=0.2)[0]
values, bins, _ = plt.hist(shuffled_selectivity_idxs, bins=bins, edgecolor='k', linewidth=1)
plt.axvline(selectivity_idx, color='r', linewidth=3)
plt.xlabel('Selectivity Index (K)')
plt.ylabel('# of runs')
plt.show()

# Tuning with circular variance
As seen in Carandini, Rose, Schumacher papers

## Circular Functions

In [ ]:
def wrap_negative(angles, bound=180):
    """Wrap angles to the range [-180, 180]"""
    bound_excess_idx = np.argwhere(angles > bound)
    out_angles = angles.copy()
    out_angles[bound_excess_idx] = (angles[bound_excess_idx] % bound) - bound
 
    return out_angles

def wrap_sort_negative(angles, values, bound=180):
    """Wrap angles to  the range [-180, 180]"""
    wrapped_angles = wrap_negative(angles)
    sorted_index = np.argsort(wrapped_angles)
    sorted_values = values[sorted_index]
    sorted_wrapped_angles = wrapped_angles[sorted_index]

    return sorted_wrapped_angles, sorted_values

def wrap_sort(angles, bound=360):
    angles_wrapped = fk.wrap(angles)
    sort_idx = np.argsort(angles_wrapped)
    angles_sorted = angles_wrapped[sort_idx]
    return angles_sorted, sort_idx

def map_statistic(x, func=np.nanmean):
    x_array = np.array(x)
    output = func(x_array, axis=1)
    return output

def get_resultant_vector(angles, tuning_weights, tuning_kind):
    
    # When calculating mean resultant, need to make orientation (bound from [0, pi]) bound from [0, 2pi]
    # See comment here: https://www.mathworks.com/matlabcentral/fileexchange/10676-circular-statistics-toolbox-directional-statistics
    if tuning_kind == 'orientation':
        multiplier = 2
    else:
        multiplier = 1
        
    mean_resultant = circ.moment(multiplier*angles,  w=tuning_weights, d=multiplier*np.mean(np.diff(angles)))
    angle = np.angle(mean_resultant) / multiplier
    length = np.absolute(mean_resultant)
    
    return length, angle

def get_circ_var(angles, tuning_weights, tuning_kind):
    
    # When calculating mean resultant, need to make orientation (bound from [0, pi]) bound from [0, 2pi]
    if tuning_kind == 'orientation':
        multiplier = 2
    else:
        multiplier = 1
        
    return circ.var(multiplier*angles, w=tuning_weights, d=multiplier*np.mean(np.diff(angles))) / multiplier

def generate_mean_response_vector(cell_responses, tuning_kind):
    # Get mean response across trials
    current_cell_mean_resp = cell_responses.groupby([tuning_kind]).apply(np.nanmean)

    # Normalize responses and fill any NaNs with zeros
    current_cell_mean_resp = current_cell_mean_resp.fillna(0)
    

    # Get a list of directions/orientations
    angles = current_cell_mean_resp.index.to_numpy()

    # Wrap angles from [-180, 180] to [0, 360] for direction tuning, and sort angles and cell responses
    # Needed for pycircstat toolbox
    angles_sorted, sort_idx = wrap_sort(angles)
    current_cell_mean_resp = current_cell_mean_resp.to_numpy()
    sorted_mean_resp = current_cell_mean_resp[sort_idx]

    # Make sure to explicitly represent 360 degrees in the data if looking at direction tuning. This is done by duplicating the 0 degresstrace
    if tuning_kind == 'direction':
        idx_0 = np.argwhere(angles_sorted == 0)[0][0]
        trace_360 = sorted_mean_resp[idx_0]
        angles_sorted = np.append(angles_sorted, 360.)
        sorted_mean_resp = np.append(sorted_mean_resp, [trace_360], axis=0)
    
    return sorted_mean_resp, angles_sorted


def generate_sem_vector(cell_responses, tuning_kind):
    # Get mean response across trials
    current_cell_resp_sem = cell_responses.groupby([tuning_kind]).apply(sem, nan_policy='omit')

    # Normalize responses and fill any NaNs with zeros
    current_cell_resp_sem = current_cell_resp_sem.fillna(0)
    
    # Get a list of directions/orientations
    angles = current_cell_resp_sem.index.to_numpy()

    # Wrap angles from [-180, 180] to [0, 360] for direction tuning, and sort angles and cell responses
    # Needed for pycircstat toolbox
    angles_sorted, sort_idx = wrap_sort(angles)
    current_cell_resp_sem = current_cell_resp_sem.to_numpy()
    sorted_sem_resp = current_cell_resp_sem[sort_idx]

    # Make sure to explicitly represent 360 degrees in the data if looking at direction tuning. This is done by duplicating the 0 degresstrace
    if tuning_kind == 'direction':
        idx_0 = np.argwhere(angles_sorted == 0)[0][0]
        trace_360 = sorted_sem_resp[idx_0]
        angles_sorted = np.append(angles_sorted, 360.)
        sorted_sem_resp = np.append(sorted_sem_resp, [trace_360], axis=0)
    
    return sorted_sem_resp, angles_sorted



def bootstrap_responsivity(trial_responses, tuning_kind, num_shuffles=1000):
    
    shuffled_responsivity = []
    
    # Get a list of directions/orientations
    angles = trial_responses.index.unique().to_numpy()
    
    trial_idxs = trial_responses.index.to_numpy()

    for i in range(0, num_shuffles):
        # Shuffle stimulus labels and reassign
        np.random.shuffle(trial_idxs)
        shuffled_responses = trial_responses.copy().reindex(trial_idxs)

        # Generate mean response vector from shuffled data
        shuffled_response_vector, angles = generate_mean_response_vector(shuffled_responses, tuning_kind)
        
         #-- Get response circular variance --#
        circ_var = get_circ_var(np.deg2rad(angles), shuffled_response_vector, tuning_kind)
        responsivity = 1 - circ_var
        shuffled_responsivity.append(responsivity)

    shuffled_responsivity = np.array(shuffled_responsivity)
    
    return shuffled_responsivity

def polar_vector_sum(magnitudes, directions):
    directions = np.deg2rad(directions)
    x = np.sum(magnitudes * np.cos(directions))
    y = np.sum(magnitudes * np.sin(directions))
    
    r = np.sqrt(x**2 + y**2)
    theta = np.rad2deg(np.arctan2(y, x))
    
    return r, theta

## Main Loop

In [ ]:
keys = ['direction', 'orientation']

# Allocate memory for cell property dataframes
df_list_circvar = []

for i, (ds, exp) in enumerate(zip(matched_data, exp_type)):

    # get the number of cells
    cells = [el for el in ds.columns if 'spikes' in el]
    
    # allocate memory for the experiment
    exp_list = []

    for key in keys:
        key_abbr = key[:3]

        # Get the mean reponse per trial
        trial_responses = ds.groupby([key, 'trial_num'])[cells].agg(np.nanmean)

        # Drop the inter-trial interval from df
        trial_responses = trial_responses.droplevel(['trial_num'])
        trial_responses = trial_responses.drop(trial_responses[trial_responses.index < -500].index)

        df_data_list = []

        for cell in cells:
            
            #-- Create the response vector --#
            current_cell_responses = trial_responses.iloc[:, trial_responses.columns.get_loc(cell)]
            mean_resp, angles = generate_mean_response_vector(current_cell_responses, key)
            sem_resp, _ = generate_sem_vector(current_cell_responses, key)
            
            # Normalize to maximum
            if np.max(mean_resp) > 0:
                norm_mean_resp = mean_resp / np.max(mean_resp)
                norm_sem_resp = sem_resp / mean_resp
            else:
                norm_mean_resp = mean_resp
                norm_sem_resp = sem_resp

            #-- Get preferred tuning --#
            resultant_length, resultant_angle = get_resultant_vector(np.deg2rad(angles), norm_mean_resp, key)
            resultant_angle = fk.wrap(np.rad2deg(resultant_angle))

            #-- Get response circular variance --#
            circ_var = get_circ_var(np.deg2rad(angles), norm_mean_resp, key)
            responsivity = 1 - circ_var
            
            # -- Run permutation test --#
            # Here we shuffle the trial IDs and compare the real selectivity index to the bootstrapped distribution
            shuffled_responsivity = bootstrap_responsivity(current_cell_responses, key, num_shuffles=300)
            p = percentileofscore(shuffled_responsivity, responsivity, kind='mean') / 100.
            
            #-- Fit Gaussian tuning curves to get preference and tuning width --#
            if key == "direction":
                fit, fit_curve, pref_angle, real_pref_angle = calculate_pref_direction(angles, norm_mean_resp)
            else:
                fit, fit_curve, pref_angle, real_pref_angle = calculate_pref_orientation(angles, norm_mean_resp)

            # Get tuning width
            tuning_width = get_FWHM(fit.x[2])
            
            # Append all the data for saving
            df_data_list.append([cell, responsivity, p, resultant_angle, pref_angle, tuning_width, (angles, norm_mean_resp), norm_sem_resp, fit_curve])

        # Make data frame for the experiment
        cell_props = pd.DataFrame(data=df_data_list, 
                                  columns=['cell', f'{key_abbr}_resp', f'{key_abbr}_resp_p', f'{key_abbr}_res_angle', f'{key_abbr}_pref', f'{key_abbr}_width', f'{key_abbr}_tuning_curve', f'{key_abbr}_sem', f'{key_abbr}_fit_curve'],
                                      )
        cell_props.insert(0, 'exp_type', ''.join([exp, str(i)]))
        exp_list.append(cell_props)
        
    df_list_circvar.append(exp_list[0].merge(exp_list[1], on=['cell', 'exp_type']))

In [ ]:
bins = np.linspace(0, max(responsivity, np.max(shuffled_responsivity)), retstep=0.2)[0]
values, bins, _ = plt.hist(shuffled_responsivity, bins=bins, edgecolor='k', linewidth=1)
plt.axvline(responsivity, color='r', linewidth=3)
plt.xlabel('1 - circ_var')
plt.ylabel('# of runs')
plt.text(0, 200, str(p)) 
plt.show()

In [ ]:
idx =  3
cell_properties_cv = df_list_circvar[0]
tuning_curve = cell_properties_cv.iloc[idx].dir_tuning_curve
fit_curve = cell_properties_cv.iloc[idx].dir_fit_curve
pref = cell_properties_cv.iloc[idx].dir_pref
tuning_width = cell_properties_cv.iloc[idx].dir_width

plot_tuning_curve(tuning_curve, fit=fit_curve, pref_angle=pref)

print(tuning_curve)

In [ ]:
# allocate memory for the cell plots
cell_plots = []

# for all the files
for ds in df_list_circvar:
    for i, cell in ds.iterrows():
        
        cell_plot = plot_tuning_curve_hv(cell.ori_tuning_curve,
                                         sem=cell.ori_sem,
                                         fit=cell.ori_fit_curve, 
                                         pref_angle=cell.ori_pref)
        
        cell_plots.append(cell_plot.opts(xlim=(0,180), xlabel="angle [deg]", title=str(cell.cell)))
        
num_cells = np.max([len(ds.cell.unique()) for ds in df_list_circvar])
hv.Layout(cell_plots).opts(shared_axes=False).cols(num_cells)

In [ ]:
# allocate memory for the cell plots
cell_plots = []

# for all the files
for ds in df_list_circvar:
    for i, cell in ds.iterrows():
        
        cell_plot = plot_tuning_curve_hv(cell.dir_tuning_curve, 
                                         sem=cell.dir_sem,
                                         fit=cell.dir_fit_curve, 
                                         pref_angle=cell.dir_pref)
        cell_plots.append(cell_plot.opts(xlim=(0, 360), xlabel="angle [deg]", title=str(cell.cell)))
        
num_cells = np.max([len(ds.cell.unique()) for ds in df_list_circvar])
hv.Layout(cell_plots).opts(shared_axes=False).cols(num_cells)

In [ ]:
df_idx = 0
this_ds = df_list_circvar[df_idx]
this_ds = this_ds.sort_values(['dir_resp'], ascending=False, inplace=False).reset_index(drop=True)

# cell_idx =  int(this_ds[this_ds.cell == 'cell_19'].index.values)
cell_idx = 13
cell_num = this_ds.iloc[cell_idx].cell
tuning_curve = this_ds.iloc[cell_idx].dir_tuning_curve
fit_curve = this_ds.iloc[cell_idx].dir_fit_curve
pref = this_ds.iloc[cell_idx].dir_pref
tuning_width = this_ds.iloc[cell_idx].dir_width
resp = this_ds.iloc[cell_idx].dir_resp
resp_p = this_ds.iloc[cell_idx].dir_resp_p

# angles, sorted_tuning = wrap_sort_negative(directions_wrapped, tuning_curve)
# sorted_fit_curve = wrap_sort_negative(*fit_curve)
# if pref > 180:
#     pref = pref % 180 - 180
# plot_tuning_curve_hv(angles, sorted_tuning, fit=sorted_fit_curve, pref_angle=pref)

fig = plot_tuning_curve_hv(tuning_curve, fit=fit_curve, pref_angle=pref)

print(cell_num, pref, tuning_width, resp, resp_p)
fig

In [ ]:
df_idx = 1
this_ds = df_list_circvar[df_idx]
this_ds = this_ds.sort_values(['ori_resp'], ascending=False, inplace=False).reset_index(drop=True)

# cell_idx =  int(this_ds[this_ds.cell == 'cell_19'].index.values)
cell_idx = 4


cell_num = this_ds.iloc[cell_idx].cell
tuning_curve = this_ds.iloc[cell_idx].ori_tuning_curve
fit_curve = this_ds.iloc[cell_idx].ori_fit_curve
pref = this_ds.iloc[cell_idx].ori_pref
tuning_width = this_ds.iloc[cell_idx].ori_width
resp = this_ds.iloc[cell_idx].ori_resp
resp_p = this_ds.iloc[cell_idx].ori_resp_p

fig = plot_tuning_curve_hv(tuning_curve, fit=fit_curve, pref_angle=pref)

print(cell_num, pref, tuning_width, resp, resp_p)
fig

In [ ]:
df_idx = 0
this_ds = df_list_circvar[df_idx]
this_ds = this_ds.sort_values(['ori_resp_p', 'dir_resp_p'], ascending=True, inplace=False).reset_index(drop=True)

# cell_idx =  int(this_ds[this_ds.cell == 'cell_0'].index.values)
cell_idx = 15
cell_num = this_ds.iloc[cell_idx].cell
dir_tuning_curve = this_ds.iloc[cell_idx].dir_tuning_curve
dir_fit_curve = this_ds.iloc[cell_idx].dir_fit_curve
dir_pref = this_ds.iloc[cell_idx].dir_pref
dir_tuning_width = this_ds.iloc[cell_idx].dir_width
dir_resp = this_ds.iloc[cell_idx].dir_resp
dir_resp_p = this_ds.iloc[cell_idx].dir_resp_p

angles, sorted_dir_tuning = wrap_sort_negative(directions_wrapped, dir_tuning_curve)
sorted_dir_fit_curve = wrap_sort_negative(*dir_fit_curve)
if dir_pref > 180:
    dir_pref = dir_pref % 180 - 180

print(cell_num, dir_pref, dir_tuning_width, dir_resp, dir_resp_p)
dir_fig = plot_tuning_curve_hv(angles, sorted_dir_tuning, fit=sorted_dir_fit_curve, pref_angle=dir_pref, xlabel='Direction [deg]', ylabel='Mean Response [inf. spikes]')

ori_tuning_curve = this_ds.iloc[cell_idx].ori_tuning_curve
ori_fit_curve = this_ds.iloc[cell_idx].ori_fit_curve
ori_pref = this_ds.iloc[cell_idx].ori_pref
ori_tuning_width = this_ds.iloc[cell_idx].ori_width
ori_resp = this_ds.iloc[cell_idx].ori_resp
ori_resp_p = this_ds.iloc[cell_idx].ori_resp_p

print(cell_num, ori_pref, ori_tuning_width, ori_resp, ori_resp_p)
ori_fig = plot_tuning_curve_hv(orientations, ori_tuning_curve, fit=ori_fit_curve, pref_angle=ori_pref, xlabel='Orientation [deg]', ylabel='Mean Response [inf. spikes]')

hv.Layout(dir_fig + ori_fig).opts(shared_axes=False)

# Compare SVD and Circ Var based metrics

In [ ]:
cell_properties_svd = df_list[-1]
# The cutoff of QI here is from the Baden paper
sig_dir_sel_svd = cell_properties_svd[(cell_properties_svd.dir_sel_p <= 0.15)].sort_values('dir_sel_p')
sig_ori_sel_svd = cell_properties_svd[(cell_properties_svd.ori_sel_p <= 0.15)].sort_values('ori_sel_p')

cell_properties_cv = df_list_circvar[-1]
sig_dir_sel_cv = cell_properties_cv[(cell_properties_cv.dir_resp > 0.25)].sort_values('dir_resp')
sig_ori_sel_cv = cell_properties_cv[(cell_properties_cv.ori_resp > 0.25)].sort_values('ori_resp')

In [ ]:
print(cell_properties_svd.columns)
print(cell_properties_cv.columns)

In [ ]:
shared_dir_sel = sig_dir_sel_svd.merge(sig_dir_sel_cv, on='cell', how='left', suffixes=['_svd', '_cv'])
shared_ori_sel = sig_ori_sel_svd.merge(sig_ori_sel_cv, on='cell', how='left', suffixes=['_svd', '_cv'])

In [ ]:
shared_dir_sel[['cell', 'dir_sel_p', 'dir_pref_svd', 'dir_pref_cv', 'dir_resp']]

In [ ]:
shared_dir_sel[['cell', 'ori_sel_p', 'ori_pref_svd', 'ori_pref_cv', 'ori_resp']]

# Population Vector Decoding

Use SVD chosen, direction selective cells

In [ ]:
cell_properties_svd = df_list[-1]
# The cutoff of QI here is from the Baden paper
sig_dir_sel_svd = cell_properties_svd[(cell_properties_svd.dir_sel_p <= 0.15)].sort_values('dir_sel_p')
cell_tuning = sig_dir_sel_svd[['cell', 'dir_pref']].sort_index()
tuned_cells = cell_tuning.cell.to_list()
dir_prefs = cell_tuning.dir_pref.to_list()

In [ ]:
print(tuned_cells)
print(dir_prefs)

In [ ]:
# Get the mean reponse per trial
trial_responses = data[-1].groupby(['trial_num'])[['direction'] + tuned_cells].agg(np.nanmean)

# Drop the inter-trial interval from df
trial_responses = trial_responses.drop(trial_responses[trial_responses.index == 0].index)

real_direction = trial_responses.direction.to_numpy()

decoded_angles = []
for i, row in trial_responses.iterrows():
    direction = row.direction
    cell_resps = row[tuned_cells]
    
    #-- Get preferred tuning --#
    decoded_length, decoded_angle = get_resultant_vector(np.deg2rad(dir_prefs), cell_resps, 'direction')
    decoded_angle = np.rad2deg(decoded_angle)
    if decoded_angle > 180:
        decoded_angle = decoded_angle % 180 - 180
    decoded_angles.append(decoded_angle)
    
decoded_angles = np.array(decoded_angles)

# for cell in cells:

#     current_cell_responses = trial_responses.iloc[:, trial_responses.columns.get_loc(cell)]

#     # Get mean response across trials
#     current_cell_mean_resp = current_cell_responses.groupby([key]).apply(np.nanmean)

#     # Normalize responses and fill any NaNs with zeros
#     current_cell_mean_resp = current_cell_mean_resp.fillna(0)

#     # Get a list of directions/orientations
#     angles = current_cell_mean_resp.index.to_numpy()

#     # Wrap angles from [-180, 180] to [0, 360] for direction tuning, and sort angles and cell responses
#     # Needed for pycircstat toolbox
#     angles_wrapped = fk.wrap(angles)
#     sort_idx = np.argsort(angles_wrapped)
#     angles_sorted = angles_wrapped[sort_idx]

#     current_cell_mean_resp = current_cell_mean_resp.to_numpy()
#     sorted_mean_resp = current_cell_mean_resp[sort_idx]

#     #-- Get preferred tuning --#
#     resultant_length, resultant_angle = get_resultant_vector(np.deg2rad(angles_sorted), sorted_mean_resp, key)
#     resultant_angle = fk.wrap(np.rad2deg(resultant_angle))


In [ ]:
print(real_direction)

In [ ]:
print(decoded_angles)

In [ ]:
real_direction - decoded_angles

In [ ]:
plt.imshow(np.cov(real_direction, decoded_angles))

# Polar plots

In [ ]:
# Test polar vector sum for direction
fig, axes = plt.subplots(nrows=len(data), ncols=len(cells), subplot_kw={'projection': 'polar'}, figsize=(20,40))

for j, ds in enumerate(data):
    cells = [el for el in ds.columns if 'cell' in el]
    tuning = ds.groupby(['direction'])[cells].mean()
    tuning_sem = ds.groupby(['direction'])[cells].sem()

    for i, cell in enumerate(cells):

        baseline = tuning.iloc[0, tuning.columns.get_loc(cell)]
        current_cell = tuning.iloc[1:, tuning.columns.get_loc(cell)] - baseline
        current_cell -= current_cell.min()
        current_cell_norm = current_cell / current_cell.max()
        current_sem = (tuning_sem.iloc[1:, tuning_sem.columns.get_loc(cell)])
        directions = current_cell.index.to_numpy()
        r, theta = polar_vector_sum(current_cell.to_numpy(), directions)

        axes[j,i].plot(np.deg2rad(directions), current_cell.to_numpy())
        axes[j,i].fill_between(np.deg2rad(directions), current_cell.to_numpy()+current_sem.to_numpy(), current_cell.to_numpy()-current_sem.to_numpy(), alpha=0.2)
        axes[j,i].plot([0, np.deg2rad(theta)], [0, current_cell.max()], color='r')
        axes[j,i].set_rmin(0)
        axes[j,i].set_theta_zero_location("W")
        axes[j,i].set_theta_direction(-1)
        axes[j,i].set_xticklabels(['0', '45', '90', '135', '+/-180', '-135', '-90', '-45'])
        axes[j,i].set_title(cell)

plt.tight_layout()

In [ ]:
# Test polar vector sum for orientation
fig, axes = plt.subplots(nrows=len(data), ncols=len(cells), subplot_kw={'projection': 'polar'}, figsize=(20,40))

for j, ds in enumerate(data):
    cells = [el for el in ds.columns if 'cell' in el]
    tuning = ds.groupby(['orientation'])[cells].mean()
    tuning_sem = ds.groupby(['orientation'])[cells].sem()

    for i, cell in enumerate(cells):

        baseline = tuning.iloc[0, tuning.columns.get_loc(cell)]
        current_cell = tuning.iloc[1:, tuning.columns.get_loc(cell)] - baseline
        current_cell -= current_cell.min()
        current_cell_norm = current_cell / current_cell.max()
        current_sem = (tuning_sem.iloc[1:, tuning_sem.columns.get_loc(cell)])
        directions = current_cell.index.to_numpy()
        r, theta = polar_vector_sum(current_cell.to_numpy(), directions)

        axes[j,i].plot(np.deg2rad(directions), current_cell.to_numpy())
        axes[j,i].fill_between(np.deg2rad(directions), current_cell.to_numpy()+current_sem.to_numpy(), current_cell.to_numpy()-current_sem.to_numpy(), alpha=0.2)
        axes[j,i].plot([0, np.deg2rad(theta)], [0, current_cell.max()], color='r')
        axes[j,i].set_rmin(0)
        axes[j,i].set_theta_zero_location("W")
        axes[j,i].set_theta_direction(-1)
        axes[j,i].set_thetamax(180)
        axes[j,i].set_title(cell)
